**Purpose**: This script simulates the solution to create the output of **Category Propensity project** as cmd_id, category, item_id (e.g: class_id, designer_id,..), item_name (e.g class_name, designer,..), propensity_score, propensity_rank, next best_rank. 

**Create date on:** Fec 11   
    Using in Category Propensity recs as Class_Propensity_als

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType,FloatType, StringType
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
998,application_1570493391423_65274,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
schema1 = StructType([StructField("cmd_id", StringType(), True),
                     StructField("category", StringType(), True),
                    StructField("propensity_score", FloatType(), True),
                    StructField("propensity_rank", IntegerType(), True)])

spark = SparkSession.builder.getOrCreate()

tmp1 = spark.createDataFrame([("001","AAA",27.4,1),("001","AAB",24.4,2),("001","ABA",18.6,3),
                             ("001","ACB",15.8,4),("002","DDB",14.1,1),("002","ABA",12.5,2)],schema1)
tmp1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+---------------+
|cmd_id|category|propensity_score|propensity_rank|
+------+--------+----------------+---------------+
|   001|     AAA|            27.4|              1|
|   001|     AAB|            24.4|              2|
|   001|     ABA|            18.6|              3|
|   001|     ACB|            15.8|              4|
|   002|     DDB|            14.1|              1|
|   002|     ABA|            12.5|              2|
+------+--------+----------------+---------------+

In [3]:
tmp1= tmp1.withColumn("id_category",F.concat(F.col('cmd_id'), F.col('category')))
tmp1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+---------------+-----------+
|cmd_id|category|propensity_score|propensity_rank|id_category|
+------+--------+----------------+---------------+-----------+
|   001|     AAA|            27.4|              1|     001AAA|
|   001|     AAB|            24.4|              2|     001AAB|
|   001|     ABA|            18.6|              3|     001ABA|
|   001|     ACB|            15.8|              4|     001ACB|
|   002|     DDB|            14.1|              1|     002DDB|
|   002|     ABA|            12.5|              2|     002ABA|
+------+--------+----------------+---------------+-----------+

In [4]:
schema2 = StructType([StructField("cmd_id", StringType(), True),
                     StructField("category", StringType(), True)])

spark = SparkSession.builder.getOrCreate()

tmp2 = spark.createDataFrame([("001","ACA"),("001","AAB"),("001","ABA"),
                             ("001","EEE"),("002","DDB"),("003","RRE")],schema2)
tmp2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+
|cmd_id|category|
+------+--------+
|   001|     ACA|
|   001|     AAB|
|   001|     ABA|
|   001|     EEE|
|   002|     DDB|
|   003|     RRE|
+------+--------+

In [5]:
tmp2= tmp2.withColumn("id_category",F.concat(F.col('cmd_id'), F.col('category')))
tmp2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-----------+
|cmd_id|category|id_category|
+------+--------+-----------+
|   001|     ACA|     001ACA|
|   001|     AAB|     001AAB|
|   001|     ABA|     001ABA|
|   001|     EEE|     001EEE|
|   002|     DDB|     002DDB|
|   003|     RRE|     003RRE|
+------+--------+-----------+

In [8]:
tmp3 = tmp1.join(tmp2 ,tmp1.id_category == tmp2.id_category, "leftanti")
tmp3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+---------------+-----------+
|cmd_id|category|propensity_score|propensity_rank|id_category|
+------+--------+----------------+---------------+-----------+
|   001|     AAA|            27.4|              1|     001AAA|
|   001|     ACB|            15.8|              4|     001ACB|
|   002|     ABA|            12.5|              2|     002ABA|
+------+--------+----------------+---------------+-----------+

In [9]:
tmp3= tmp3.selectExpr('cmd_id','category','propensity_score')
tmp3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+
|cmd_id|category|propensity_score|
+------+--------+----------------+
|   001|     AAA|            27.4|
|   001|     ACB|            15.8|
|   002|     ABA|            12.5|
+------+--------+----------------+

In [10]:
# Re_order after merge
tmp3 = tmp3.orderBy(F.col("cmd_id"),F.col("propensity_score").desc())
tmp3 = tmp3.selectExpr("cmd_id", "category","propensity_score")
tmp3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+
|cmd_id|category|propensity_score|
+------+--------+----------------+
|   001|     AAA|            27.4|
|   001|     ACB|            15.8|
|   002|     ABA|            12.5|
+------+--------+----------------+

In [11]:
from pyspark.sql.window import Window

window = Window.partitionBy(tmp3['cmd_id']).orderBy(tmp3['propensity_score'].desc())
tmp3 =tmp3.select('*', F.rank().over(window).alias('next_best_rank')).filter(F.col('next_best_rank') <= 5) 

tmp3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+--------------+
|cmd_id|category|propensity_score|next_best_rank|
+------+--------+----------------+--------------+
|   001|     AAA|            27.4|             1|
|   001|     ACB|            15.8|             2|
|   002|     ABA|            12.5|             1|
+------+--------+----------------+--------------+

In [12]:
tmp1 = tmp1.selectExpr('cmd_id','category','propensity_score','propensity_rank')
tmp1 =tmp1.join(tmp3.selectExpr('cmd_id','category','next_best_rank'),['cmd_id','category'],how='left')
tmp1 = tmp1.orderBy(F.col("cmd_id"),F.col("propensity_rank"))
tmp1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+----------------+---------------+--------------+
|cmd_id|category|propensity_score|propensity_rank|next_best_rank|
+------+--------+----------------+---------------+--------------+
|   001|     AAA|            27.4|              1|             1|
|   001|     AAB|            24.4|              2|          null|
|   001|     ABA|            18.6|              3|          null|
|   001|     ACB|            15.8|              4|             2|
|   002|     DDB|            14.1|              1|          null|
|   002|     ABA|            12.5|              2|             1|
+------+--------+----------------+---------------+--------------+